In [ ]:
# 코랩에서 실행시 먼저 설치해야 한다
!pip install JPype1
!pip install konlpy
!pip install pyMySQL
!pip install -U finance-datareader

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
# 리스트 flatten을 위한 itertools 패키지 import
import itertools
import warnings
warnings.filterwarnings('ignore')

#* 1. 데이터 수집
- 1.1 주가 OHLCV 데이터
  - 샘플 종목: 005930
- 1.2 뉴스데이터

In [ ]:
# 주가 데이터 확인
# DB 연결 준비
conn = sqlite3.connect('/content/nlp_sample.db')
# 종목 지정:  stock_code 005930
scode = '005930'
# DB에서 종목 데이터  추출위한 sql 준비
sql = f"SELECT * FROM A{scode} "
# DB 검색결과를 dataframe에 저장
df_stock_price = pd.read_sql_query(sql, conn)
# DB close
conn.close()

# 실행확인을 위한 화면 출력
print(df_stock_price.head())

#* 뉴스 데이터
- 네이버 증권, 실시간 속보
- beautifulsoup 사용

In [ ]:
# 뉴스 데이터 수집부터
# 크롤링 할 웹주소 URL
news_url= f"https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258&date=20230823&page=13"
# 웹 request 위한 html 형식 설정
html = requests.get(news_url)
# BeautifulSoup 으로 해당 웹페이지 읽어오기
soup = BeautifulSoup(html.text, "lxml")

In [ ]:
# 실행확인을 위한 화면 출력
print(soup)

In [ ]:
# 가져올 뉴스가 있는 부분 확인
# <dd class='articleSummary'> 태그 목록 만들기
ddsum_list = soup.find_all('dd', attrs={"class": "articleSummary"})

In [ ]:
# 실행확인을 위한 화면 출력
ddsum_list[0]

In [ ]:
# 뉴스만 읽어오기
# 현재 페이지 전체 뉴스 저장을 위한 news_df 생성
news_df = pd.DataFrame()
# 뉴스 있는 동안 반복
if len(ddsum_list)>0:
  # <dd> 태그 목록에서 하나씩 꺼내기
  for ddsum in ddsum_list:
    # <dd> 와 </dd> 사이의 텍스트를 추출, 다른 태그 있을시 '^'구분해서 추출, 앞뒤 공백제거
    data_string = ddsum.getText('^').strip()
    # 추출한 문자열을 '^' 경계로 분리
    string_list = data_string.split('^')
    # 2줄 요약 news_summary 문자열 추출
    news_summary  = string_list[0].strip()
    # 신문사 press_name 문자열 추출
    press_name    = string_list[1].strip()
    # 배포날짜 release_date 문자열 추출
    release_date  = string_list[5].strip()
    # 추출한 위 3 항목을 DataFrame으로 저장
    temp_df = pd.DataFrame(data = [[news_summary,press_name,release_date]], columns=['summary','press','rdate'])
    # 현재 페이지 전체 뉴스를 위한 news_df에 추가
    news_df = pd.concat([news_df, temp_df], ignore_index=True)

In [ ]:
# 실행 확인을 위한 화면 출력
news_df

In [ ]:
# 웹크롤링으로 가져온 뉴스를 DB에 저장하기 & 꺼내오기
#==================================================================
# 뉴스를 DB에 저장하기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
news_df.to_sql('news_4train', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#==================================================================

#==================================================================
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_4train"
# sql 실행 결과 데이터프레임으로 받기
df_from_db = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#==================================================================

In [ ]:
# 실행 확인 위한 화면 출력
print("From DB: ", df_from_db.head())

2. 학습 모델을 위한 데이터 가공
- 1) 주가 데이터 레이블링
- 2) 뉴스 자연어 처리

In [ ]:
# 주가 데이터 레이블링
#==================================================================
# DB에서 주가 데이터 읽어 오기
# DB 연결 준비
conn = sqlite3.connect('/content/nlp_sample.db')
# 종목 지정:  stock_code 041190
scode = '005930'
# DB에서 종목 데이터  추출위한 sql 준비
sql = f"SELECT * FROM A{scode} "
# DB 검색결과를 dataframe에 저장
df_stock_price = pd.read_sql_query(sql, conn)
# DB close
conn.close()
#==================================================================

In [ ]:
# 실행확인을 위한 화면 출력
print(df_stock_price.head())

In [ ]:
#==================================================================
# 레이블링한 테이블 만들기 위한 df 준비
df_bun_labeled= df_stock_price.copy()
# 분당 변동률도 구해서 넣고
df_bun_labeled['bun_rate'] = ((df_stock_price['close']-df_stock_price['open'])/df_stock_price['open'])*100
# 레이블도 넣기: Up,Hold,Down
df_bun_labeled.loc[df_bun_labeled['bun_rate']>0,'bun_label'] = 'U'
df_bun_labeled.loc[df_bun_labeled['bun_rate']==0,'bun_label'] = 'H'
df_bun_labeled.loc[df_bun_labeled['bun_rate']<0,'bun_label'] = 'D'
# DB저장 위한 컬럼 정리
df_bun_labeled = df_bun_labeled[['date','bun_rate','bun_label']]
#==================================================================

In [ ]:
# 실행 확인을 위한 화면 출력
print(df_bun_labeled.head())

In [ ]:
#==================================================================
# 레이블링한 결과 DB에 저장하기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
df_bun_labeled.to_sql('A005930_bun_label', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#==================================================================

#==================================================================
# 레이블링한 결과 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM A005930_bun_label"
# sql 실행 결과 데이터프레임으로 받기
df_from_db = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#==================================================================

In [ ]:
# 실행 확인을 위한 화면 출력
print(df_from_db.head())

2.2 뉴스 자연어 처리
  - 2.2.1 konlpy 커스터마이징
  - 2.2.2 뉴스 토큰화
  - 2.2.3 뉴스 단어사전 만들기
  - 2.2.4 뉴스 벡터화


In [ ]:
##########################################################
# konlpy 커스터마이징 테스트 코드
# 삼성전자 -> '삼성', '전자'(X), '삼성전자'(O) 확인
#--------------------------------------------------------
okt = Okt()

print(okt.pos("삼성전자 005930 새로닉스 두산에너빌리티 "))
print(okt.pos("에코프로 "))
print(okt.pos("제주반도체 현대힘스 패스트파이브"))
print(okt.pos("아이오아이 르세라핌"))
#########################################################

In [ ]:
#=====================================================================
# --------------------------------------------------------------------
# Konlpy에 한국거래소 상장종목 전체 종목명과 약명을 새로운 단어로 추가하기
# --------------------------------------------------------------------
# konlpy 디렉토리로 이동
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
# 현재 디렉토리 확인
os.getcwd()
# 압축 풀 임시 디렉토리 /aaa 만들기
os.makedirs('./aaa')
# 임시디렉토리로 이동
os.chdir('./aaa')
# 압축 풀기
!jar xvf ../open-korean-text-2.1.0.jar
# FinanceDataReader로 한국거래소 상장종목 전체 종목을 가져오기
df_krx = fdr.StockListing('KRX')
# 종목명만 추출하기
name_list = df_krx['Name']
# 새로운 단어목록 생성을 위한 data 변수 선언
data = ''
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 새로운 단어로 넣기
# DB 연결 준비: 종목명 약어 읽어오기
conn = pymysql.connect(host= '34.64.198.150', port = 3306, user="big17", password="big1743211", db="team1_project", charset = 'utf8')
# DB에서 종목명 약어 추출위한 sql 준비
sql = f"SELECT x.kor_name,x.kor_abb FROM team1_project.stock_name_abb x"
# DB 검색결과를 dataframe에 저장
name_abb_df = pd.read_sql_query(sql, conn)
# 실행확인을 위한 화면 출력
print(name_abb_df.head(3))
# DB close
conn.close()
# 종목명 추가하기
name_list = name_abb_df['kor_name']
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 추가하기
name_list = name_abb_df['kor_abb']
# 종목명 약명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# konlpy의 company_names.txt 사전으로 저장
with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/aaa/org/openkoreantext/processor/util/noun/company_names.txt", 'w') as f:
    f.write(data)
# 임시 작업 디렉토리로 이동 확인
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java/aaa')
# 다시 압축
!jar cvf ../open-korean-text-2.1.0.jar *
#------------------------------------------------------------------
# 코랩 실행시 Restart session 실행
# import도 다시 실행
#==================================================================

In [ ]:
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
# 리스트 flatten을 위한 itertools 패키지 import
import itertools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
##########################################################
# konlpy 커스터마이징 테스트 코드
# 삼성전자 -> '삼성', '전자'(X), '삼성전자'(O) 확인
#--------------------------------------------------------
okt = Okt()

print(okt.pos("삼성전자 005930 새로닉스 두산에너빌리티"))
print(okt.pos("에코프로 "))
print(okt.pos("제주반도체 현대힘스 패스트파이브"))
print(okt.pos("아이오아이 르세라핌"))
#########################################################

In [ ]:
#====================================================================
# 뉴스 토큰화 하기
#-------------------------------------------------------------------
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_4train"
# sql 실행 결과 데이터프레임으로 받기
df_news_4train = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()

In [ ]:
# 실행확인을 위한 화면 출력
print(df_news_4train.head(3))

In [ ]:
#-------------------------------------------------------------------
# 토큰화를 위한 konlpy의 Okt 객체 생성
okt=Okt()
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_4train['morphs_tokenized_summary'] = df_news_4train['summary'].apply(okt.morphs)
# DB 저장을 위해 문자열로 변환
df_news_4train['morphs_tokenized_summary'] = df_news_4train['morphs_tokenized_summary'].astype(str)
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_4train['pos_tokenized_summary'] = df_news_4train['summary'].apply(okt.pos)
# DB 저장을 위해 문자열로 변환
df_news_4train['pos_tokenized_summary'] = df_news_4train['pos_tokenized_summary'].astype(str)
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_4train['nouns_tokenized_summary'] = df_news_4train['summary'].apply(okt.nouns)
# DB 저장을 위해 문자열로 변환
df_news_4train['nouns_tokenized_summary'] = df_news_4train['nouns_tokenized_summary'].astype(str)
# pos tokenized 에서 Modifier,Josa,Suffix,Punctuation,Foreigh 빼기, 1 자리 단어 빼기
df_news_4train['tokenized_summary'] = ""
# 모든 뉴스처리
for i in range(len(df_news_4train)):
  # 토큰화된 뉴스 임시 저장
  new_str = []
  # 형태소 분리된 내용을 튜플로 꺼내기
  for (x,y) in ast.literal_eval(df_news_4train['pos_tokenized_summary'][i]):
    #관형사, 조사, 접두어, 문장부호, 한자등 제외
    if (y!='Modifier')&(y!='Josa')&(y!='Suffix')&(y!='Punctuation')&(y!='Foreign'):
      # 1자리 단어도 제외
      if(len(str(x))>1):
        # 남은 것 모두 붙이기
        new_str.append(x)
  #토큰화 결과 컬럼에 넣기
  df_news_4train['tokenized_summary'][i]=str(new_str)
#-------------------------------------------------------------------

In [ ]:
# 실행확인을 위한 화면 출력
df_news_4train.head(3)

In [ ]:
#===================================================================
# 토큰화한 결과 DB 저장하기
#DB 저장을 위해서 필요한 컬럼 정리
df_news_4train = df_news_4train[['summary','press','rdate','tokenized_summary','pos_tokenized_summary']]
# 실행 확인을 위한 화면 출력
#print(result_df.head(3))
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
df_news_4train.to_sql('news_4train', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#===================================================================

#===================================================================
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_4train"
# sql 실행 결과 데이터프레임으로 받기
df_news_from_db = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#===================================================================

In [ ]:
# 실행확인을 위한 화면 출력
print(df_news_from_db.head(3))

In [ ]:
#===================================================================
# 단어사전 생성: 2만단어 단어사전 만들기: TextVectorization 커스터마이즈
#-------------------------------------------------------------------
# custom_표준화 함수
def custom_standardization_fn(string_tensor):
    # 한글의 경우 소문자 처리 안해도 됨
    lowercase_string = string_tensor
    # 문장부호중에 '%','.' 는 남기기
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation.replace('%','').replace('.',''))}]", "")

# custom_토큰화 함수
def custom_split_fn(string_tensor):
    # 한글의 경우 형태소 분석 처리 해야 한다: 우린 이미 okt로 했음
    return tf.strings.split(string_tensor)

# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기
text_vectorization = TextVectorization(
    #--- max_tokens 지정--
    max_tokens=20000
    #---------------------
    # 정수 인덱스로 출력
  ,  output_mode="int"
    # 표준화 함수를 custom_표준화 함수로 지정
  ,  standardize=custom_standardization_fn
    # 토큰화 함수를 custom_토큰화 함수로 지정
  ,  split=custom_split_fn
)


In [ ]:
# 사전만들기 위해 토큰화된 text를 넣을 dataset 리스트 생성
dataset = []
# 전체 data 반복
for i in range (len(df_news_from_db)):
  # 토큰화된 text 를 하나의 리스트로 만든다
  dataset.append(eval(df_news_from_db['tokenized_summary'][i]))
#  중첩된 리스트를 flatten 시킨다
dataset = list(itertools.chain(*dataset))

# ============================================
# .adapt()로 단어 사전 만들기
text_vectorization.adapt(dataset)
# ============================================

# 어휘사전 확인 .get_vocabulary() 함수
len(text_vectorization.get_vocabulary())

In [ ]:
# =======================================================
# 단어사전 pickle로 저장
# --- 저장 : start ---
# Vector for word "this"
print (text_vectorization("삼성전자"))
print (text_vectorization("LG"))
print (text_vectorization("미국"))

# Pickle the config and weights
pickle.dump({'config': text_vectorization.get_config(),
             'weights': text_vectorization.get_weights()}
            , open("tv_2man.pkl", "wb"))

# Later you can unpickle and use
# `config` to create object and
# `weights` to load the trained weights.
# --- 저장 : end ---
# =========================================================

In [ ]:
# =========================================================
# 단어사전 불러오기
# --- 불러오기 load : start ---
from_disk = pickle.load(open("tv_2man.pkl", "rb"))

new_vectorizer = TextVectorization(
    # 정수 인덱스로 출력
    output_mode="int",
    # 표준화 함수를 custom_표준화 함수로 지정
    standardize=custom_standardization_fn,
    # 토큰화 함수를 custom_토큰화 함수로 지정
    split=custom_split_fn,
  )
new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_vectorizer.set_weights(from_disk['weights'])

# Lets see the Vector for word "this"
print (text_vectorization("삼성전자"))
print (text_vectorization("LG"))
print (text_vectorization("미국"))
# --- 불러오기 load : end ---
# =============================================================

In [ ]:
# 벡터화 전
df_news_from_db.head()

In [ ]:
# =============================================================
# 뉴스 벡터화 하기
# 토큰화된 것을 사전이용해서 벡터화 하고 텐서안의 인덱스 리스트를 꺼내서 따로 저장
for i in range(len(df_news_from_db)):
  df_news_from_db.at[i,'20000_encoded_text_code']= str(new_vectorizer(df_news_from_db.at[i,'tokenized_summary']).numpy()).replace('\n', '')
# =============================================================

In [ ]:
# 벡터화 후
df_news_from_db.head()

In [ ]:
#===================================================================
# 벡터화 결과 DB 저장하기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
df_news_from_db.to_sql('news_4train', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#===================================================================
#===================================================================
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_4train"
# sql 실행 결과 데이터프레임으로 받기
df_news_from_db = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#===================================================================

In [ ]:
df_news_from_db.head(10)

##* 3. 데이터셋 만들기
* 뉴스를 분단위로 flatten 하기
* 뉴스 + 주가 데이터 레이블 연결
* - 종목 분봉 데이터를 뉴스 속보 기간에 맞춰서 자르기 : 20230823
* 양쪽 테이블에 join을 위한 공통 컬럼 만들기
- left join으로 레이블 넣기

In [ ]:
#---------------------------------------------------------------
# 뉴스 분단위 Flatten 하기
# 작업할 복사본 만들기
df_work = df_news_from_db.copy()

# 문자열 형식을 datetime형식으로 변환
df_work['date_rdate']= pd.to_datetime(df_work['rdate'])

# 하루치 flatten된 뉴스 속보 저장을 위한 df 생성
df_work_result = pd.DataFrame()

# 시작시간: 0시0분
c_time = pd.to_datetime(df_news_from_db.at[len(df_news_from_db)-1,'rdate'])
# 종료시간: 23시59분
e_time = pd.to_datetime(df_news_from_db.at[0,'rdate'])

# 당일 0시0분 부터 23시59분까지 처리
while c_time <= e_time:
    # 1분씩 증가
    n_time = c_time +pd.Timedelta(minutes=1)
    # 실행 확인용 화면 출력
    #print('-',c_time, n_time)
    # 같은 시간대(분 단위)에 나온 속보 모두 추출
    df_sametime= df_work.loc[(df_work['date_rdate']>=c_time) & (df_work['date_rdate']<n_time),['date_rdate','20000_encoded_text_code']]
    # 뉴스 속보 벡터열 넣을 문자열 변수 생성, 초기화
    f_string=''
    # 같은 시간대 나온 모든 뉴스 속보 벡터를 하나로 합치기 flatten
    if len(df_sametime)>=1:     # 뉴스가 1개 이상 존재하는 경우
        for i in range(len(df_sametime)):   # 뉴스 갯수 만큼 반복
            # 실행 확인용 화면 출력
            #print('--',i)
            # 벡터 숫자만 꺼내기: 양 끝 기호 [ ] 없애고 중간에 있는 \n삭제
            t_str = df_sametime.iloc[i]['20000_encoded_text_code'].strip('[]').replace('\n','')
            # 앞 뉴스 벡터열에 이어서 붙이기, 공백 ' ' 을 사이에 두고
            f_string = f_string +' '+ t_str
            # 혹시 공백 여러개 있으면 하나로 정리해서 넣기
            f_string = re.sub(' +', ' ', f_string)
        # 같은 시간대 뉴스 벡터 모두 합쳤으면 앞 뒤로 [ ] 기호 붙이기
        f_string = '['+f_string+']'
        # 실행 확인용 화면 출력
        #print(f_string)
    else:   # 뉴스가 없는 시간대(분 단위)의 경우
        # 속보 벡터열을 빈 문자열로 처리
        f_string=''
    # 저장을 위한 임시 df 생성, 초기화: 현재 시간, 뉴스 갯수, 벡터 스트링
    temp_df = pd.DataFrame({'rdate':[c_time],'news_cnt':[len(df_sametime)],'2000_encoded_text_code':[f_string]})
    # 하루치 flatten된 뉴스 속보 저장을 위한 결과에 추가
    df_work_result= pd.concat([df_work_result,temp_df], ignore_index=True)
    # 다음 시간(분단위)을 현재 시간으로 설정: 반복 위해서
    c_time=n_time
#----------------------------------------------------------------

In [ ]:
# 실행 확인을 위한 화면 출력
df_work_result.tail()

In [ ]:
#--------------------------------------------------------
# Flatten 결과 DB에 저장하기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
df_work_result.to_sql('news_4train_flatten', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#--------------------------------------------------------

#--------------------------------------------------------
# Flatten 결과 DB에서 가져오기
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_4train_flatten"
# sql 실행 결과 데이터프레임으로 받기
df_news_from_db = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#--------------------------------------------------------

In [ ]:
# DB 연결 준비
conn = sqlite3.connect('nlp_sample.db')
# 종목 지정:  stock_code 041190
scode = '005930'
# DB에서 종목 데이터  추출위한 sql 준비
sql = f"SELECT * FROM A{scode}_bun_label "
# DB 검색결과를 dataframe에 저장
stock_df = pd.read_sql_query(sql, conn)
# DB close
conn.close()

# 실행확인을 위한 화면 출력
print('주가 시작:',stock_df.at[0,'date'])
print('주가   끝:',stock_df.at[len(stock_df)-1,'date'])

print('뉴스 시작:',df_news_from_db.at[len(df_news_from_db)-1,'rdate'])
print('뉴스   끝:',df_news_from_db.at[0,'rdate'])


In [ ]:
# 안전한 날짜 연산을 위해서 datetime 형태의 날짜 컬럼을 만든다
stock_df['tdate'] = pd.to_datetime(stock_df['date'].astype(str))

In [ ]:
# 실행확인을 위한 화면 출력
stock_df.head()

In [ ]:
# 전체 주가 데이터 길이
len(stock_df)

190702

In [ ]:
# 뉴스 당일과 그 다음날 주가 정보만 잘라두기
stock_df = stock_df[(stock_df['tdate'].dt.date >= pd.to_datetime(df_news_from_db.at[len(df_news_from_db)-1,'rdate']).date())
                    &(stock_df['tdate'].dt.date <= pd.to_datetime(df_news_from_db.at[len(df_news_from_db)-1,'rdate']).date()+pd.Timedelta(days=1))
                    ]

In [ ]:
# 뉴스 당일과 그 다음날 주가 정보 갯수
len(stock_df)

762

In [ ]:
#----------------------------------------------------
#시작시간부터 끝 시간까지 1분 단위로 행 만들어서 주가 레이블 정보 끼워 넣기
stock_df.sort_values('tdate', ascending=False, inplace=True, ignore_index=True)
stock_df.head()
# df 복사
temp_df = stock_df.copy()
# 시작시간:
c_time= temp_df.iloc[0]['tdate']
# 종료시간:
e_time = temp_df.iloc[-1]['tdate']

# 날짜 범위 생성
date_range = pd.date_range(start=e_time, end=c_time, freq='1min')

# 데이터프레임 생성
left_temp_df = pd.DataFrame({'tdate': date_range})
# 역순 정렬
left_temp_df.sort_values('tdate', ascending=False, ignore_index=True, inplace=True)

lj_result_df = pd.merge(left_temp_df, temp_df, on='tdate', how='left')
#print(lj_result_df.head(20))
# bun_rate와 bun_label이 Nan 인 경우 앞 시간의 값 rate, label을 넣는다
lj_result_df['bun_rate'] = lj_result_df['bun_rate'].fillna(method='ffill')
lj_result_df['bun_label'] = lj_result_df['bun_label'].fillna(method='ffill')
#print(lj_result_df.head(10))
stock_df=lj_result_df.copy()
#----------------------------------------------------
# 컬럼명 수정: 종목번호 추가
stock_df.rename(columns={'date':'005930_date','bun_rate': '005930_bun_rate','bun_label': '005930_bun_label'}, inplace=True)

# 해당 날짜 뉴스 테이블 읽어오기
one_news_df= df_work_result.copy()
# one_news_df에 tdate 만들기, 20230102 날짜 하루 뉴스
one_news_df['tdate']=pd.to_datetime(one_news_df['rdate'])
# 뉴스 속보 테이블에 종목 분별 레이블 join 시켜서 넣기(left join)
left_join_result_df=pd.merge(one_news_df, stock_df, on='tdate', how='left')

In [ ]:
# 실행 확인을 위한 화면 출력
left_join_result_df

In [ ]:
#====================================================================
# 데이터 셋으로 정리하기
df_dataset = left_join_result_df[['2000_encoded_text_code','005930_bun_label']]
df_dataset = df_dataset[df_dataset['2000_encoded_text_code'] != '']
df_dataset.columns=['text','label']
# 레이블 숫자로 변환 return
df_dataset['label'] = df_dataset['label'].replace({'U': 2, 'H': 1, 'D': 0})
#====================================================================

In [ ]:
# 실행 확인을 위한 화면 출력
df_dataset

#* 4. 예측 모델 만들기
- **LSTM**

In [ ]:
#---------------------------------------------------------------
# 예측 모델 만들기: LSTM 카테고리 분류 모델
#-- import packages --------------------------------------------
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import reuters
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
#----------------------------------------------------------------
# 4.1 데이터셋 로딩
# data load: 데이터X,레이블y 분리
X = df_dataset['text']
y = df_dataset['label']

# 안전한 데이터 로딩을 위해서 문자열 형식인 데이터X를 리스트로 확인 변환
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])

# 레이블이 1/0 아닌 경우: 원핫 인코딩 처리
y = to_categorical(y)

# data set separation: 학습셋, 테스트셋 분리
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)

# 레이블 카테고리 지정
category = 3

# 실행 확인을 위한 화면 출력
print(category,'카테고리')
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')

# pad_sequence 단어 수 맞춰주기: 한 기사당 단어 갯수 제한
sequence_length = 50
X_train = sequence.pad_sequences(X_train, maxlen= sequence_length)
X_test = sequence.pad_sequences(X_test,maxlen=sequence_length)

####################################################################################
#-----------------------------------------------------------------------------------
# 4.2 모델 생성
# model creation: LSTM
model = Sequential()
model.add(Embedding(20000,sequence_length))         # Input : 임베딩(2만단어사전, 50단어)
model.add(LSTM(sequence_length, activation='tanh')) # LSTM  : 활성화: tanh
model.add(Dense(category, activation='softmax'))    # Output: ternary 이상인 경우

#-------------------------------------------------------------
# 4.3 모델 컴파일
# model compile, 실행 옵션
model.compile(
    optimizer = 'adam'                    # 최적화 함수 : adam
  , loss = 'categorical_crossentropy'     # 손실 함수   : categorical_crossentropy(ternary 이상인 경우)
  , metrics = ['accuracy']                # 측정 지표   : 정확도 accuracy
)

# 조기중단 설정, early stopping callback
early_stopping_callback = EarlyStopping(
    monitor= 'val_loss'                   # 모니터 항목 : val_loss
  , patience = 10                          # patiencd    : 5 (epochs)
)

# 최적화 모델 저장, best model saving checkpoint
modelpath = f'./data/model/LSTM_news.hdf5' # 모델저장위치와 이름

# checkpointer 생성
checkpointer = ModelCheckpoint(
    filepath = modelpath        # filepath
  , monitor = 'val_loss'        # 모니터  : val_loss
  , verbose = 0                 # verbose : 0
  , save_best_only = True       # save_best_only: True
)
#-------------------------------------------------------------
# 4.4 모델 핏
# model fit, 모델 학습 실행, history 저장
history = model.fit(
        X_train                     # 학습 데이터
    ,   y_train                     # 레이블
    ,   batch_size = 4              # batch_size  : 4
    ,   epochs = 10                 # epochs      : 10
    ,   validation_data = (X_test, y_test)    # validation_data
    ,   callbacks = [               # callbacks
            early_stopping_callback # 조기중단
          , checkpointer            # best model
        ]
)

#-------------------------------------------------------------
# 4.5 모델 테스트
# 시각화
# 테스트 정확도 출력
print(f'\n Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}')

# 검증셋과 학습셋의 오차를 저장
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프 그리기
# x축: 실행한 epochs 횟수
x_len = np.arange(len(y_loss))
# 선그래프: 테스트셋 손실, red
plt.plot(x_len, y_vloss, marker='.',c='red',label='Testset_loss')
# 선그래프: 학습셋 손실, blue
plt.plot(x_len, y_loss, marker='.',c='blue', label='Trainset_loss')
# 그래프 legend 추가, 위치 지정
plt.legend(loc='upper right')
# 그래프에 그리드 표시
plt.grid()
# x 축 레이블
plt.xlabel('epoch')
# y 축 레이블
plt.ylabel('loss')
# 그래프 화면에 출력
plt.show()
#--------------------------------------------------------------
# 예측모델 결과 저장 확인하기!
#==============================================================

#5. 예측 하기


1. 예측 데이터
2. 예측 모델 load
3. 예측 실행
4. 예측 결과

In [ ]:
#----------------------------------------------------------------
# 예측 데이터 가져오기
# 데이터 전처리
# 모델 입력 데이터 셋 생성
# 1.웹 크롤링: 오늘 날짜 뉴스 가져다가 예측(분류) 해보기!
news_url= f"https://finance.naver.com/news/news_list.naver?mode=LSS2D&section_id=101&section_id2=258"
# 웹 request 위한 html 형식 설정
html = requests.get(news_url)
# BeautifulSoup 으로 해당 웹페이지 읽어오기
soup = BeautifulSoup(html.text, "lxml")
# <dd class='articleSummary'> 태그 목록 만들기
ddsum_list = soup.find_all('dd', attrs={"class": "articleSummary"})
# 태그들을 '^'구분해서 분리
ddsum_test_list = ddsum_list[0].getText('^').strip().split('^')
# 2줄 요약 news_summary 문자열 추출
news_summary  = ddsum_test_list[0].strip()
# 신문사 press_name 문자열 추출
press_name    = ddsum_test_list[1].strip()
# 배포날짜 release_date 문자열 추출
release_date  = ddsum_test_list[5].strip()
# 추출한 위 3 항목을 DataFrame으로 저장
temp_df = pd.DataFrame(data = [[news_summary,press_name,release_date]], columns=['summary','press','rdate'])
# 실행 확인용 화면 출력
print('오늘: summary',temp_df['summary'].values)
print('오늘: press',temp_df['press'].values)
print('오늘: rdate',temp_df['rdate'].values)
# 현재 페이지 전체 뉴스 저장을 위한 news_df 생성
news_df = pd.DataFrame()
# 뉴스 있는 동안 반복
if len(ddsum_list)>0:
  # <dd> 태그 목록에서 하나씩 꺼내기
  for ddsum in ddsum_list:
    # <dd> 와 </dd> 사이의 텍스트를 추출, 다른 태그 있을시 '^'구분해서 추출, 앞뒤 공백제거
    data_string = ddsum.getText('^').strip()
    # 추출한 문자열을 '^' 경계로 분리
    string_list = data_string.split('^')
    # 2줄 요약 news_summary 문자열 추출
    news_summary  = string_list[0].strip()
    # 신문사 press_name 문자열 추출
    press_name    = string_list[1].strip()
    # 배포날짜 release_date 문자열 추출
    release_date  = string_list[5].strip()
    # 추출한 위 3 항목을 DataFrame으로 저장
    temp_df = pd.DataFrame(data = [[news_summary,press_name,release_date]], columns=['summary','press','rdate'])
    # 현재 페이지 전체 뉴스를 위한 news_df에 추가
    news_df = pd.concat([news_df, temp_df], ignore_index=True)
#==================================================================
# 뉴스를 DB에 저장하기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
news_df.to_sql('news_today', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#==================================================================
#==================================================================
# 뉴스를 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_today"
# sql 실행 결과 데이터프레임으로 받기
df_news_today = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#==================================================================
# 뉴스토큰화
#-------------------------------------------------------------------
# 토큰화를 위한 konlpy의 Okt 객체 생성
okt=Okt()
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_today['morphs_tokenized_summary'] = df_news_today['summary'].apply(okt.morphs)
# DB 저장을 위해 문자열로 변환
df_news_today['morphs_tokenized_summary'] = df_news_today['morphs_tokenized_summary'].astype(str)
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_today['pos_tokenized_summary'] = df_news_today['summary'].apply(okt.pos)
# DB 저장을 위해 문자열로 변환
df_news_today['pos_tokenized_summary'] = df_news_today['pos_tokenized_summary'].astype(str)
# okt로 tokenize해서 형태소pos 추출하기 (tokenize 전체는 okt.morphs, 명사는 okt.nouns)
df_news_today['nouns_tokenized_summary'] = df_news_today['summary'].apply(okt.nouns)
# DB 저장을 위해 문자열로 변환
df_news_today['nouns_tokenized_summary'] = df_news_today['nouns_tokenized_summary'].astype(str)
# pos tokenized 에서 Modifier,Josa,Suffix,Punctuation,Foreigh 빼기, 1 자리 단어 빼기
df_news_today['tokenized_summary'] = ""
# 모든 뉴스처리
for i in range(len(df_news_today)):
  # 토큰화된 뉴스 임시 저장
  new_str = []
  # 형태소 분리된 내용을 튜플로 꺼내기
  for (x,y) in ast.literal_eval(df_news_today['pos_tokenized_summary'][i]):
    #관형사, 조사, 접두어, 문장부호, 한자등 제외
    if (y!='Modifier')&(y!='Josa')&(y!='Suffix')&(y!='Punctuation')&(y!='Foreign'):
      # 1자리 단어도 제외
      if(len(str(x))>1):
        # 남은 것 모두 붙이기
        new_str.append(x)
  #토큰화 결과 컬럼에 넣기
  df_news_today['tokenized_summary'][i]=str(new_str)
#-------------------------------------------------------------------
#===================================================================
# 커스터마이즈된 TextVectorization 객체 생성
#-------------------------------------------------------------------
# custom_표준화 함수
def custom_standardization_fn(string_tensor):
    # 한글의 경우 소문자 처리 안해도 됨
    lowercase_string = string_tensor
    # 문장부호중에 '%','.' 는 남기기
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation.replace('%','').replace('.',''))}]", "")

# custom_토큰화 함수
def custom_split_fn(string_tensor):
    # 한글의 경우 형태소 분석 처리 해야 한다: 우린 이미 okt로 했음
    return tf.strings.split(string_tensor)

# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기
text_vectorization = TextVectorization(
    #--- max_tokens 지정--
    max_tokens=20000
    #---------------------
    # 정수 인덱스로 출력
  ,  output_mode="int"
    # 표준화 함수를 custom_표준화 함수로 지정
  ,  standardize=custom_standardization_fn
    # 토큰화 함수를 custom_토큰화 함수로 지정
  ,  split=custom_split_fn
)
# =========================================================
# 단어사전 불러오기(이미 만들어진 사전)
# --- 불러오기 load : start ---
from_disk = pickle.load(open("tv_2man.pkl", "rb"))

new_vectorizer = TextVectorization(
    # 정수 인덱스로 출력
    output_mode="int",
    # 표준화 함수를 custom_표준화 함수로 지정
    standardize=custom_standardization_fn,
    # 토큰화 함수를 custom_토큰화 함수로 지정
    split=custom_split_fn,
  )
new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_vectorizer.set_weights(from_disk['weights'])
# --- 불러오기 load : end ---
# =============================================================

# =============================================================
# 뉴스 벡터화 하기
# 토큰화된 것을 사전이용해서 벡터화 하고 텐서안의 인덱스 리스트를 꺼내서 따로 저장
for i in range(len(df_news_today)):
  df_news_today.at[i,'20000_encoded_text_code']= str(new_vectorizer(df_news_today.at[i,'tokenized_summary']).numpy()).replace('\n', '')
# =============================================================
# 실행 확인용 화면 출력
print(df_news_today.head())

# 예측 데이터 X_test 생성
X = df_news_today['20000_encoded_text_code']
# 벡터 리스트형태도 다시 확인 생성
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])
# LSTM 모델 입력을 위한 패딩 맞추기
sequence_length = 50
# 입력을 위한 최종 sequence 형태 맞추기
X = sequence.pad_sequences(X, maxlen= sequence_length)
#-------------------------------------------------------------------
# 예측 데이터 처리 끝
#===================================================================

In [ ]:
#======================================================================
# 예측 모델 로딩, 실행, 결과
#------------------------------------------------------------------------------
from tensorflow.keras.models import Model, Sequential,load_model

# 예측 모델 load
model = load_model(filepath=f'/content/data/model/LSTM_news.hdf5')
# 예측 실행
stock_outlook = np.argmax(model.predict(X), axis=1)
# 예측 결과 저장
df_news_today['outlook'] = stock_outlook
#-----------------------------------------------------------------------------------

1/1 [==============================] - 1s 585ms/step


In [ ]:
# 실행 확인을 위한 화면 출력
df_news_today[['summary','outlook']]

In [ ]:
#--------------------------------------------------------
# 예측 결과 DB에 저장
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 데이터프레임을 SQLite 테이블로 저장
df_news_today.to_sql('news_today_outlook', conn, if_exists='replace', index=False)
# 데이터베이스 연결 종료
conn.close()
#---------------------------------------------------------
#---------------------------------------------------------
# 예측 결과 DB에서 꺼내오기
# DB 커넥션 생성
conn = sqlite3.connect('nlp_sample.db')
# 읽어오는 SQL
query = "SELECT * FROM news_today_outlook"
# sql 실행 결과 데이터프레임으로 받기
df_news_today_outlook = pd.read_sql_query(query, conn)
# 데이터베이스 연결 종료
conn.close()
#---------------------------------------------------------

In [ ]:
# 실행 확인용 화면 출력
print(df_news_today_outlook.head())